In [1]:
import requests
import re
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import json

import pathlib
import sys

parent_dir = pathlib.Path.cwd().parent
sys.path.append(str(parent_dir))

# Getting Bills

In [2]:
from olascrape import summarize

In [3]:
with open("../documents.json") as f: 
    docs = json.load(f)

In [4]:
summaries = summarize(docs)

with open("summaries.json", "w") as f:
    json.dump(summaries, f)

In [8]:
len(summaries.keys()) == len(docs.keys())

True

In [10]:
summaries["bill-174"]

'bill-174 -- Bill 174 | Title: Supply Act, 2024 | Sponsor: Mulroney, Hon. Caroline President of the Treasury Board | Current status: Royal Assent received. Statutes of Ontario 2024, chapter 4'

In [11]:
with open("summaries.json") as f:
    test = json.load(f)

In [13]:
test.keys()

dict_keys(['bill-1', 'bill-2', 'bill-3', 'bill-4', 'bill-5', 'bill-6', 'bill-7', 'bill-8', 'bill-9', 'bill-10', 'bill-11', 'bill-12', 'bill-13', 'bill-14', 'bill-15', 'bill-16', 'bill-17', 'bill-18', 'bill-19', 'bill-20', 'bill-21', 'bill-22', 'bill-23', 'bill-24', 'bill-25', 'bill-26', 'bill-27', 'bill-28', 'bill-29', 'bill-30', 'bill-31', 'bill-32', 'bill-33', 'bill-34', 'bill-35', 'bill-36', 'bill-37', 'bill-38', 'bill-39', 'bill-40', 'bill-41', 'bill-42', 'bill-43', 'bill-44', 'bill-45', 'bill-46', 'bill-47', 'bill-48', 'bill-49', 'bill-50', 'bill-51', 'bill-52', 'bill-53', 'bill-54', 'bill-55', 'bill-56', 'bill-57', 'bill-58', 'bill-59', 'bill-60', 'bill-61', 'bill-62', 'bill-63', 'bill-64', 'bill-65', 'bill-66', 'bill-67', 'bill-68', 'bill-69', 'bill-70', 'bill-71', 'bill-72', 'bill-73', 'bill-74', 'bill-75', 'bill-76', 'bill-77', 'bill-78', 'bill-79', 'bill-80', 'bill-81', 'bill-82', 'bill-83', 'bill-84', 'bill-85', 'bill-86', 'bill-87', 'bill-88', 'bill-89', 'bill-90', 'bill-91